### 0. Imports

In [2858]:
import pandas as pd

# work with environment variables
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve database credentials
USERNAME = os.getenv("DATABASE_USERNAME")
PASSWORD = os.getenv("DATABASE_PASSWORD")

# append parent folder to path
import sys
sys.path.append("..")

# import data load support functions
import src.data_load_support as dls

# import database connection support functions
import src.database_connection_support as dcs

# 1. Introduction to this notebook

## 1.1 Load tables

In [3067]:
weather = pd.read_parquet("../data/weather/transformed/weather.parquet")

# 2. Database design

In [2860]:
airports = pd.read_parquet("../data/airport_codes/transformed/countries_airports.parquet")
airports.head()

,country,city_name,city_entityid,airport_skyid,airport_entityid,airport_name,latitude,longitude
0,spain,madrid,27544850,MAD,95565077,madrid,40.416705,-3.703582
1,spain,barcelona,27548283,BCN,95565085,barcelona,41.382894,2.177432
2,spain,port of spain,27546011,POS,104120358,port of spain,NaN,NaN
3,spain,malaga,27547484,AGP,95565095,malaga,36.721303,-4.421637
4,spain,seville,27547022,SVQ,95565089,seville,37.388630,-5.995340


## 2.1 Cities

```sql
-- Create cities table
CREATE TABLE cities (
    city_entityid SERIAL PRIMARY KEY,
    city_name VARCHAR(255) NOT NULL,
    country VARCHAR(100),
    latitude NUMERIC,
    longitude NUMERIC,
);
```

In [2861]:
cities = airports[["country","city_name","city_entityid","latitude","longitude"]]

## 2.2 Flights

### 2.2.1 Flight tables - Airports

```sql
-- Create airports table
CREATE TABLE airports (
    airport_entityid SERIAL PRIMARY KEY,
    airport_skyid VARCHAR(10) NOT NULL,
    airport_name VARCHAR(255) NOT NULL,
    city_entityid INT REFERENCES cities(city_entityid) ON DELETE SET NULL
);
```

- airport_entityid PK
- airport_skyid
- airport_name
- city_entityid


In [2862]:
airports = airports[["airport_entityid","airport_skyid","airport_name","city_entityid"]]

### 2.2.2 Flight tables - Flights

In [2863]:
itineraries = pd.read_parquet("../data/flights/transformed/itineraries.parquet")
itineraries.head()

,itinerary_id,query_date,score,duration,price,price_currency,stops,departure,arrival,company,...,fare_is_change_allowed,fare_is_partially_changeable,fare_is_cancellation_allowed,fare_is_partially_refundable,origin_airport,destination_airport,origin_airport_code,destination_airport_code,origin_airport_entityid,destination_airport_entityid
0,13870-2411081510--32680-0-9772-2411081635,2024-11-04 05:52:24.873861,0.999000,85,49,€,0,2024-11-08 15:10:00,2024-11-08 16:35:00,Air Europa,...,False,False,False,False,Madrid,Barcelona,MAD,BCN,95565077,95565085
1,13870-2411080715--32222-0-9772-2411080830,2024-11-04 05:52:24.879866,0.864733,75,38,€,0,2024-11-08 07:15:00,2024-11-08 08:30:00,Iberia,...,False,False,False,False,Madrid,Barcelona,MAD,BCN,95565077,95565085
2,13870-2411081455--32222-0-9772-2411081610,2024-11-04 05:52:24.881888,0.707501,75,63,€,0,2024-11-08 14:55:00,2024-11-08 16:10:00,Iberia,...,False,False,False,False,Madrid,Barcelona,MAD,BCN,95565077,95565085
3,13870-2411080745--32222-0-9772-2411080900,2024-11-04 05:52:24.881888,0.553361,75,40,€,0,2024-11-08 07:45:00,2024-11-08 09:00:00,Iberia,...,False,False,False,False,Madrid,Barcelona,MAD,BCN,95565077,95565085
4,13870-2411080830--32222-0-9772-2411080945,2024-11-04 05:52:24.883025,0.551953,75,47,€,0,2024-11-08 08:30:00,2024-11-08 09:45:00,Iberia,...,False,False,False,False,Madrid,Barcelona,MAD,BCN,95565077,95565085


- itinerary_id PK
- origin_airport_entityid FK
- destination_airport_entityid FK
- departure_datetime
- arrival_datetime
- company
- self_transfer
- fare_is_change_allowed
- fare_is_partially_changeable
- fare_is_cancellation_allowed
- fare_is_partially_refundable


In [2864]:
itineraries

,itinerary_id,query_date,score,duration,price,price_currency,stops,departure,arrival,company,...,fare_is_change_allowed,fare_is_partially_changeable,fare_is_cancellation_allowed,fare_is_partially_refundable,origin_airport,destination_airport,origin_airport_code,destination_airport_code,origin_airport_entityid,destination_airport_entityid
0,13870-2411081510--32680-0-9772-2411081635,2024-11-04 05:52:24.873861,0.999000,85,49,€,0,2024-11-08 15:10:00,2024-11-08 16:35:00,Air Europa,...,False,False,False,False,Madrid,Barcelona,MAD,BCN,95565077,95565085
1,13870-2411080715--32222-0-9772-2411080830,2024-11-04 05:52:24.879866,0.864733,75,38,€,0,2024-11-08 07:15:00,2024-11-08 08:30:00,Iberia,...,False,False,False,False,Madrid,Barcelona,MAD,BCN,95565077,95565085
2,13870-2411081455--32222-0-9772-2411081610,2024-11-04 05:52:24.881888,0.707501,75,63,€,0,2024-11-08 14:55:00,2024-11-08 16:10:00,Iberia,...,False,False,False,False,Madrid,Barcelona,MAD,BCN,95565077,95565085
3,13870-2411080745--32222-0-9772-2411080900,2024-11-04 05:52:24.881888,0.553361,75,40,€,0,2024-11-08 07:45:00,2024-11-08 09:00:00,Iberia,...,False,False,False,False,Madrid,Barcelona,MAD,BCN,95565077,95565085
4,13870-2411080830--32222-0-9772-2411080945,2024-11-04 05:52:24.883025,0.551953,75,47,€,0,2024-11-08 08:30:00,2024-11-08 09:45:00,Iberia,...,False,False,False,False,Madrid,Barcelona,MAD,BCN,95565077,95565085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16547,"17543-2510260600--31781,-32222-1-13870-2510261010",2024-11-04 05:52:38.196143,0.223096,250,123,€,1,2025-10-26 06:00:00,2025-10-26 10:10:00,TAP Air Portugal,...,False,False,False,False,Valencia,Madrid,VLC,MAD,95565090,95565077
16548,17543-2510261250--32478-1-13870-2510262000,2024-11-04 05:52:38.196143,0.194155,430,388,€,1,2025-10-26 12:50:00,2025-10-26 20:00:00,Brussels Airlines,...,False,False,False,False,Valencia,Madrid,VLC,MAD,95565090,95565077
16549,17543-2510261950--31799-1-13870-2510270915,2024-11-04 05:52:38.197142,0.137538,805,467,€,1,2025-10-26 19:50:00,2025-10-27 09:15:00,SWISS,...,False,False,False,False,Valencia,Madrid,VLC,MAD,95565090,95565077
16550,17543-2510261240--31799-1-13870-2510270915,2024-11-04 05:52:38.197142,0.091166,1235,467,€,1,2025-10-26 12:40:00,2025-10-27 09:15:00,SWISS,...,False,False,False,False,Valencia,Madrid,VLC,MAD,95565090,95565077


In [2865]:
flights = itineraries[["itinerary_id","origin_airport_entityid","destination_airport_entityid","departure","arrival","company","self_transfer","fare_is_change_allowed","fare_is_partially_changeable", "fare_is_cancellation_allowed","fare_is_partially_refundable"]]

### 2.2.3 Flight tables - Flight_prices

- price_id PK
- itinerary_id FK
- query_date
- price
- price_currency
- score

In [2866]:
flight_prices = itineraries[["itinerary_id","query_date","price","price_currency","score"]]

## 2.2 Accommodations

In [2949]:
accommodations_df = pd.read_parquet("../data/accommodations/transformed/booking.parquet")
accommodations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22980 entries, 0 to 22979
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   query_date               22980 non-null  datetime64[ns]
 1   city                     22980 non-null  object        
 2   checkin                  22980 non-null  datetime64[ns]
 3   checkout                 22980 non-null  datetime64[ns]
 4   n_adults_search          22980 non-null  object        
 5   n_children_search        22980 non-null  object        
 6   n_rooms_search           22980 non-null  object        
 7   name                     22980 non-null  object        
 8   url                      22980 non-null  object        
 9   price_currency           22980 non-null  object        
 10  total_price_amount       22980 non-null  object        
 11  distance_city_center_km  22980 non-null  float64       
 12  score                    22306 n

Booking places

- place_id PK
- city_name FK
- name
- url
- distance_city_center_km
- score
- n_comments
- close_to_metro
- sustainability_cert
- location_score

In [2950]:
booking_places = accommodations_df[["city","name","url","distance_city_center_km","score","n_comments","close_to_metro","sustainability_cert","location_score"]]

Accommodations

- accommodation_id PK
- place_id PK
- room_type
- standardized_room_type
- double_bed
- single_bed
- shared_bathroom
- balcony

In [2951]:
accommodations = accommodations_df[["city","name","room_type","standardized_room_type","double_bed","single_bed","shared_bathroom","balcony"]]

Prices
- price_id PK
- accommodation_id FK
- query_date
- checkin
- checkout
- n_adults
- n_children
- n_rooms
- price_night
- price_currency
- free_cancellation
- pay_at_hotel
- free_taxi

In [2952]:
accommodations_df.columns

Index(['query_date', 'city', 'checkin', 'checkout', 'n_adults_search',
       'n_children_search', 'n_rooms_search', 'name', 'url', 'price_currency',
       'total_price_amount', 'distance_city_center_km', 'score', 'n_comments',
       'close_to_metro', 'sustainability_cert', 'room_type', 'double_bed',
       'single_bed', 'free_cancellation', 'breakfast_included', 'pay_at_hotel',
       'location_score', 'free_taxi', 'standardized_room_type',
       'shared_bathroom', 'balcony', 'price_night'],
      dtype='object')

In [2953]:
accommodation_prices = accommodations_df[["city","name","room_type","query_date","checkin","checkout","n_adults_search","n_children_search","n_rooms_search","price_night","price_currency","free_cancellation","pay_at_hotel","free_taxi"]]

## 2.3 Activities and availability

In [3041]:
activities = pd.read_parquet("../data/activities/transformed/activities.parquet")
availabilities = pd.read_parquet("../data/activities/transformed/availabilities.parquet")
activities.head(3)

,query_date,city,activity_date_range_start,activity_date_range_end,activity_name,description,url,image,available_days,available_times,duration,latitude,longitude,price,currency,category,spanish,address
0,2024-11-04 01:23:44.031007,barcelona,2024-11-05,2024-11-11,Excursión a Montserrat + Visita a una bodega,En esta excursión a Montserrat no solo disfrut...,www.civitatis.com/es/barcelona/tour-tapas-vino...,www.civitatis.com/f/espana/barcelona/tour-tapa...,"[06, 11, 07, 09, 08, 05, 10]","[[8:45], [8:45], [8:45], [8:45], [8:45], [8:45...",7h 30m,41.3940236912484,2.181866082214644,19.98,EUR,Gastronomía y enoturismo,Español,"Barcelona (Bus Terminal Nord), Eixample, 08013..."
1,2024-11-04 01:23:44.034541,barcelona,2024-11-05,2024-11-11,Paseo en catamarán al atardecer con música en ...,Contempla el skyline de Barcelona mientras dis...,www.civitatis.com/es/barcelona/paseo-catamaran...,www.civitatis.com/f/espana/barcelona/paseo-cat...,"[07, 08, 06, 11, 10, 09, 05]","[[16:30], [16:30], [17:00], [16:30], [16:30], ...",1h 30m,41.37495867288118,2.17849589524371,7.65,EUR,Paseos en barco,Español,"Moll de les Drassanes, 3P, Ciutat Vella, 08039..."
2,2024-11-04 01:23:44.037536,barcelona,2024-11-05,2024-11-11,Free tour por el Parque Güell,En este free tour por el Parque Güell conocere...,www.civitatis.com/es/barcelona/free-tour-parqu...,www.civitatis.com/f/espana/barcelona/free-tour...,"[11, 05, 06, 07, 08]","[[11:30], [11:30], [11:30], [11:30], [11:30]]",1h 30m,41.41508351,2.154768947,2.00,EUR,Visitas guiadas y free tours,Español,"Ctra. del Carmel, 23, Horta-Guinardó, 08024 Ba..."


In [3042]:
availabilities.head(3)

,query_date,available_times,city,activity_name,activity_date_range_start,activity_date_range_end,available_date
0,2024-11-04 01:23:44.031007,08:45:00,barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11,2024-11-06
1,2024-11-04 01:23:44.031007,08:45:00,barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11,2024-11-11
2,2024-11-04 01:23:44.031007,08:45:00,barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11,2024-11-07


The activites have persistent information in their name, description, url, image, duration, latitude, longitude, category, spanish (language) and address. Thus, it makes for one single table of the database.

### Activities table

- activity_id PK
- activity_name
- city
- description
- url
- image
- duration
- latitude
- longitude
- category
- spanish
- address

In [3043]:
activities_table = activities[["activity_name","city","description","url","image","duration","latitude","longitude","category","spanish","address"]]

Then, there is more time dependant informations about them, which are:
- Available date and time
- Price

Both of these are bound to query time, as prices and availabilities are subject to change, due to possible offers or changes in demand. following the rules of normalisation, they represent 2 separate tables to avoid duplicities in information.

#### Activities_prices

The price_id will be serial.

- price_id PK
- activity_id FK
- query_date
- price
- currency

The table will need the activity name and city to check for the activity_id in the database, and then map the ids to the table to upload it all.

If streamlined, it will have to do that activity by activity.

In [3044]:
activities_prices = activities[["activity_name","city","query_date","price","currency"]]

#### Activities_availabilities

- schedule_id PK
- activity_id FK
- query_date
- available_date
- available_time

In [3045]:
availabilities

,query_date,available_times,city,activity_name,activity_date_range_start,activity_date_range_end,available_date
0,2024-11-04 01:23:44.031007,08:45:00,barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11,2024-11-06
1,2024-11-04 01:23:44.031007,08:45:00,barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11,2024-11-11
2,2024-11-04 01:23:44.031007,08:45:00,barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11,2024-11-07
3,2024-11-04 01:23:44.031007,08:45:00,barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11,2024-11-09
4,2024-11-04 01:23:44.031007,08:45:00,barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11,2024-11-08
...,...,...,...,...,...,...,...
56149,2024-11-04 01:24:04.201953,08:00:00,valencia,Tour por Valencia y la Albufera + Campos de arroz,2025-10-25,2025-10-31,2025-10-27
56150,2024-11-04 01:24:04.201953,08:00:00,valencia,Tour por Valencia y la Albufera + Campos de arroz,2025-10-25,2025-10-31,2025-10-25
56151,2024-11-04 01:24:04.201953,08:00:00,valencia,Tour por Valencia y la Albufera + Campos de arroz,2025-10-25,2025-10-31,2025-10-29
56152,2024-11-04 01:24:04.201953,08:00:00,valencia,Tour por Valencia y la Albufera + Campos de arroz,2025-10-25,2025-10-31,2025-10-26


In [3046]:
activities_schedules = availabilities[["activity_name","city","query_date","available_date","available_times"]]

Cities

# 3. Database creation

## 3.1 Database creation - database

In [2878]:
database_credentials = {"username":USERNAME,
                        "password":PASSWORD,
                        }

In [ ]:
dls.create_db("travel_planner_v2", credentials_dict=database_credentials)

Database already existant.


## 3.2 Database creation - tables

In [2880]:
drop_tables = "DROP TABLE IF EXISTS cities, airports, flights, flight_prices, booking_places, accommodations, accommodation_prices, activities, activity_prices, activity_availabilities, weather_data CASCADE;"

create_cities = """
CREATE TABLE cities (
    city_entityid SERIAL PRIMARY KEY,
    city_name VARCHAR(255) NOT NULL,
    country VARCHAR(100),
    latitude NUMERIC,
    longitude NUMERIC
);
"""

create_airports = """
CREATE TABLE airports (
    airport_entityid SERIAL PRIMARY KEY,
    airport_skyid VARCHAR(10) NOT NULL,
    airport_name VARCHAR(255) NOT NULL,
    city_entityid INT REFERENCES cities(city_entityid) ON DELETE SET NULL
);
"""

create_flights = """
CREATE TABLE flights (
    itinerary_id VARCHAR(255) PRIMARY KEY,
    origin_airport_entityid INT REFERENCES airports(airport_entityid),
    destination_airport_entityid INT REFERENCES airports(airport_entityid),
    departure_datetime TIMESTAMP NOT NULL,
    arrival_datetime TIMESTAMP NOT NULL,
    company VARCHAR(100),
    self_transfer BOOLEAN,
    fare_is_change_allowed BOOLEAN,
    fare_is_partially_changeable BOOLEAN,
	fare_is_cancellation_allowed BOOLEAN,
	fare_is_partially_refundable BOOLEAN
);
"""

create_flight_prices = """
CREATE TABLE flight_prices (
    price_id SERIAL PRIMARY KEY,
    itinerary_id VARCHAR(255) REFERENCES flights(itinerary_id),
    query_date TIMESTAMP NOT NULL,
    price NUMERIC NOT NULL,
    price_currency VARCHAR(10) NOT NULL DEFAULT 'EUR',
    score NUMERIC
);
"""

create_booking_places = """
CREATE TABLE booking_places (
    place_id SERIAL PRIMARY KEY,
    city_entityid INT REFERENCES cities(city_entityid) ON DELETE SET NULL,
    name VARCHAR(255) NOT NULL,
    url TEXT,
    distance_city_center_km NUMERIC,
    score NUMERIC,
    n_comments INT,
    close_to_metro BOOLEAN,
    sustainability_cert BOOLEAN,
    location_score NUMERIC
);
"""


create_accommodations = """
CREATE TABLE accommodations (
    accommodation_id SERIAL PRIMARY KEY,
    place_id INT REFERENCES booking_places(place_id),
    room_type TEXT,
    standardized_room_type TEXT,
    double_bed BOOLEAN,
    single_bed BOOLEAN,
    shared_bathroom BOOLEAN,
    balcony BOOLEAN
);
"""

create_accommodation_prices = """
CREATE TABLE accommodation_prices (
    price_id SERIAL PRIMARY KEY,
    accommodation_id INT REFERENCES accommodations(accommodation_id),
    query_date TIMESTAMP NOT NULL,
    checkin DATE NOT NULL,
    checkout DATE NOT NULL,
    n_adults INT NOT NULL,
    n_children INT DEFAULT 0,
    n_rooms INT NOT NULL,
    price_night NUMERIC NOT NULL,
    price_currency VARCHAR(4) NOT NULL,
    free_cancellation BOOLEAN,
    pay_at_hotel BOOLEAN,
    free_taxi BOOLEAN
);
"""

create_activities = """
CREATE TABLE activities (
    activity_id SERIAL PRIMARY KEY,
    activity_name VARCHAR(500) NOT NULL,
    city_entityid INT REFERENCES cities(city_entityid) ON DELETE SET NULL,
    description TEXT,
    url VARCHAR(500),
    image VARCHAR(500),
    duration VARCHAR(30),
    latitude NUMERIC,
    longitude NUMERIC,
    category VARCHAR(100),
    spanish VARCHAR(30),
    address VARCHAR(500)
);
"""

create_activity_prices = """
CREATE TABLE activity_prices (
    price_id SERIAL PRIMARY KEY,
    activity_id INT REFERENCES activities(activity_id) ON DELETE CASCADE,
    query_date DATE NOT NULL,
    price NUMERIC NOT NULL,
    currency VARCHAR(4) NOT NULL DEFAULT 'EUR'
);
"""

create_activity_availabilities = """
CREATE TABLE activity_availabilities (
    schedule_id SERIAL PRIMARY KEY,
    activity_id INT REFERENCES activities(activity_id) ON DELETE CASCADE,
    query_date TIMESTAMP NOT NULL,
    available_date DATE NOT NULL,
    available_time TIME
);
"""

create_weather = """
CREATE TABLE weather_data (
    date DATE NOT NULL,
    apparent_temperature_mean NUMERIC,
    apparent_temperature_min NUMERIC NOT NULL,
    apparent_temperature_max NUMERIC NOT NULL,
    precipitation_sum NUMERIC,
    precipitation_hours NUMERIC NOT NULL,
    wind_speed_10m_max NUMERIC NOT NULL,
    wind_gusts_10m_max NUMERIC NOT NULL,
    sunshine_duration NUMERIC,
    daylight_duration NUMERIC NOT NULL,
    city_entityid INT REFERENCES cities(city_entityid) ON DELETE SET NULL,
    forecast_history VARCHAR(10) NOT NULL
);
"""


In [2881]:
create_table_queries = [
    drop_tables,
    create_cities,
    create_airports,
    create_flights,
    create_flight_prices,
    create_booking_places,
    create_accommodations,
    create_accommodation_prices,
    create_activities,
    create_activity_prices,
    create_activity_availabilities,
    create_weather
]

In [ ]:
conn = dcs.connect_to_database("travel_planner_v2", credentials_dict=database_credentials, autocommit=True)

# drop all tables and create:
for query in create_table_queries:
    conn.cursor().execute(query)

## 3.3 Database creation - batch table insert

### 3.3.1 Cities

In [2883]:
create_cities = """
CREATE TABLE cities (
    city_entityid SERIAL PRIMARY KEY,
    city_name VARCHAR(255) NOT NULL,
    country VARCHAR(100)
);
"""

In [2884]:
cities.head(3)

,country,city_name,city_entityid,latitude,longitude
0,spain,madrid,27544850,40.416705,-3.703582
1,spain,barcelona,27548283,41.382894,2.177432
2,spain,port of spain,27546011,NaN,NaN


In [2885]:
conn.cursor().executemany(
"INSERT INTO cities (country, city_name, city_entityid,latitude,longitude) VALUES (%s,%s,%s,%s,%s)",
list(cities.itertuples(index=False, name=None))
)

### 3.3.2 Airports

In [2886]:
create_airports = """
CREATE TABLE airports (
    airport_entityid SERIAL PRIMARY KEY,
    airport_skyid VARCHAR(10) NOT NULL,
    airport_name VARCHAR(255) NOT NULL,
    city_entityid INT REFERENCES cities(city_entityid) ON DELETE SET NULL
);
"""

In [2887]:
airports.head(3)

,airport_entityid,airport_skyid,airport_name,city_entityid
0,95565077,MAD,madrid,27544850
1,95565085,BCN,barcelona,27548283
2,104120358,POS,port of spain,27546011


In [2888]:
conn.cursor().executemany(
"INSERT INTO airports (airport_entityid, airport_skyid, airport_name, city_entityid) VALUES (%s,%s,%s,%s)",
list(airports.itertuples(index=False, name=None))
)

### 3.3.3 Flights

In [2889]:
create_flights = """
CREATE TABLE flights (
    itinerary_id VARCHAR(255) PRIMARY KEY,
    origin_airport_entityid INT REFERENCES airports(airport_entityid),
    destination_airport_entityid INT REFERENCES airports(airport_entityid),
    departure_datetime TIMESTAMP NOT NULL,
    arrival_datetime TIMESTAMP NOT NULL,
    company VARCHAR(100),
    self_transfer BOOLEAN,
    fare_is_change_allowed BOOLEAN,
    fare_is_partially_changeable BOOLEAN,
	fare_is_cancellation_allowed BOOLEAN,
	fare_is_partially_refundable BOOLEAN
);
"""

In [2890]:
flights.head(3)

,itinerary_id,origin_airport_entityid,destination_airport_entityid,departure,arrival,company,self_transfer,fare_is_change_allowed,fare_is_partially_changeable,fare_is_cancellation_allowed,fare_is_partially_refundable
0,13870-2411081510--32680-0-9772-2411081635,95565077,95565085,2024-11-08 15:10:00,2024-11-08 16:35:00,Air Europa,False,False,False,False,False
1,13870-2411080715--32222-0-9772-2411080830,95565077,95565085,2024-11-08 07:15:00,2024-11-08 08:30:00,Iberia,False,False,False,False,False
2,13870-2411081455--32222-0-9772-2411081610,95565077,95565085,2024-11-08 14:55:00,2024-11-08 16:10:00,Iberia,False,False,False,False,False


In [2891]:
conn.cursor().executemany(
    """
    INSERT INTO flights (
        itinerary_id, 
        origin_airport_entityid, 
        destination_airport_entityid, 
        departure_datetime, 
        arrival_datetime, 
        company, 
        self_transfer, 
        fare_is_change_allowed, 
        fare_is_partially_changeable, 
        fare_is_cancellation_allowed, 
        fare_is_partially_refundable
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """,
    list(flights.itertuples(index=False, name=None))
)

### 3.3.4 Flight prices

In [2892]:
create_flight_prices = """
CREATE TABLE flight_prices (
    price_id SERIAL PRIMARY KEY,
    itinerary_id VARCHAR(255) REFERENCES flights(itinerary_id),
    query_date TIMESTAMP NOT NULL,
    price NUMERIC NOT NULL,
    price_currency VARCHAR(10) NOT NULL,
    score NUMERIC
);
"""

In [2893]:
flight_prices.head(3)

,itinerary_id,query_date,price,price_currency,score
0,13870-2411081510--32680-0-9772-2411081635,2024-11-04 05:52:24.873861,49,€,0.999000
1,13870-2411080715--32222-0-9772-2411080830,2024-11-04 05:52:24.879866,38,€,0.864733
2,13870-2411081455--32222-0-9772-2411081610,2024-11-04 05:52:24.881888,63,€,0.707501


In [2894]:
conn.cursor().executemany(
    """
    INSERT INTO flight_prices (
        itinerary_id, 
        query_date, 
        price, 
        price_currency, 
        score
    ) VALUES (%s, %s, %s, %s, %s)
    """,
    list(flight_prices.itertuples(index=False, name=None))
)

### 3.3.5 Booking places

In [2895]:
create_booking_places = """
CREATE TABLE booking_places (
    place_id SERIAL PRIMARY KEY,
    city_entityid INT REFERENCES cities(city_entityid) ON DELETE SET NULL,
    name VARCHAR(255) NOT NULL,
    url VARCHAR(500),
    distance_city_center_km NUMERIC,
    score NUMERIC,
    n_comments INT,
    close_to_metro BOOLEAN,
    sustainability_cert BOOLEAN,
    location_score NUMERIC
);
"""

In [2896]:
booking_places.head(3)

,city,name,url,distance_city_center_km,score,n_comments,close_to_metro,sustainability_cert,location_score
0,barcelona,Nice and comfortable room for your stay in BCN,https://www.booking.com/hotel/es/nice-and-comf...,1.1,6.0,2,True,False,None
1,barcelona,Hotel Derby,https://www.booking.com/hotel/es/derby.es.html...,2.4,8.2,2476,True,False,None
2,barcelona,Travelodge Barcelona Poblenou,https://www.booking.com/hotel/es/travelodge-ba...,2.9,7.2,10136,True,False,None


Map city entity id

In [2897]:
city_entityid_map = cities.set_index("city_name")["city_entityid"].to_dict()

In [2898]:
booking_places["city"] = booking_places["city"].map(city_entityid_map)

In [2899]:
booking_places.head(2)

,city,name,url,distance_city_center_km,score,n_comments,close_to_metro,sustainability_cert,location_score
0,27548283,Nice and comfortable room for your stay in BCN,https://www.booking.com/hotel/es/nice-and-comf...,1.1,6.0,2,True,False,None
1,27548283,Hotel Derby,https://www.booking.com/hotel/es/derby.es.html...,2.4,8.2,2476,True,False,None


In [2900]:
booking_places.loc[:,["n_comments","location_score","score","distance_city_center_km"]] = booking_places[["n_comments","location_score","score","distance_city_center_km"]].astype(float)

In [2901]:
create_booking_places = """
CREATE TABLE booking_places (
    place_id SERIAL PRIMARY KEY,
    city_entityid INT REFERENCES cities(city_entityid) ON DELETE SET NULL,
    name VARCHAR(255) NOT NULL,
    url VARCHAR(500),
    distance_city_center_km NUMERIC,
    score NUMERIC,
    n_comments INT,
    close_to_metro BOOLEAN,
    sustainability_cert BOOLEAN,
    location_score NUMERIC
);
"""

In [2902]:
booking_places.head(2)

,city,name,url,distance_city_center_km,score,n_comments,close_to_metro,sustainability_cert,location_score
0,27548283,Nice and comfortable room for your stay in BCN,https://www.booking.com/hotel/es/nice-and-comf...,1.1,6.0,2,True,False,NaN
1,27548283,Hotel Derby,https://www.booking.com/hotel/es/derby.es.html...,2.4,8.2,2476,True,False,NaN


In [2903]:
conn.cursor().executemany(
    """
    INSERT INTO booking_places (
        city_entityid, 
        name, 
        url, 
        distance_city_center_km, 
        score,
        n_comments,
        close_to_metro,
        sustainability_cert,
        location_score
    ) VALUES (%s, %s, %s, %s, %s,%s, %s, %s, %s)
    """,
    list(booking_places.itertuples(index=False, name=None))
)

### 3.3.6 Accommodations 

Get place_id mapping

In [2904]:
def fetch_ids_from_db(conn, df, target_table, target_field, matching_fields):
    """
    Fetch IDs from the target_table based on matching_fields in the DataFrame.

    Parameters:
    - conn: Database connection object
    - df: DataFrame containing the fields to match
    - target_table: The name of the table to query (e.g., 'booking_places')
    - target_field: The field to retrieve from the target_table (e.g., 'place_id')
    - matching_fields: A dictionary where keys are field names in the target_table and values are column names in the df

    Returns:
    - A list of IDs retrieved from the database
    """
    cursor = conn.cursor()
    ids = []
    
    # Construct WHERE clause based on matching fields
    where_clause = " AND ".join([f"{table_field} = %s" for table_field in matching_fields.keys()])
    query = f"SELECT {target_field} FROM {target_table} WHERE {where_clause};"
    
    for _, row in df.iterrows():
        values = tuple(row[df_field] for df_field in matching_fields.values())
        
        cursor.execute(query, values)
        result = cursor.fetchone()
        ids.append(result[0] if result else None)

    cursor.close()
    return ids


In [2905]:
accommodations["city"] = accommodations["city"].map(city_entityid_map)

In [2906]:
matching_fields = {
    "city_entityid": "city",  # field in the database : field in the DataFrame
    "name": "name"
}

accommodations.loc[:,'place_id'] = fetch_ids_from_db(conn, accommodations, "booking_places", "place_id", matching_fields)


In [2907]:
accommodations = accommodations.iloc[:,2:]

In [2908]:
accommodations.head()

,room_type,standardized_room_type,double_bed,single_bed,shared_bathroom,balcony,place_id
0,Habitación Doble Económica,Habitación Doble Económica,True,False,False,False,1
1,Habitación (1 o 2 adultos) - 1 o 2 camas,Habitación (1 o 2 adultos) - 1 o 2 camas,True,True,False,False,2
2,Habitación Doble,Habitación Doble,True,False,False,False,3
3,Habitación Doble - 2 camas,Habitación Doble - 2 camas,False,True,False,False,4
4,Habitación Doble con baño privado,Habitación Doble con baño privado,True,False,False,False,5


In [2909]:
conn.cursor().executemany(
    """
    INSERT INTO accommodations ( 
        room_type, 
        standardized_room_type, 
        double_bed,
        single_bed,
        shared_bathroom,
        balcony,
        place_id
    ) VALUES (%s, %s, %s, %s, %s,%s, %s)
    """,
    list(accommodations.itertuples(index=False, name=None))
)

### 3.3.7 Accommodation prices

In [2910]:
def fetch_multiple_ids_from_db(conn, df, target_table, target_fields, matching_fields):
    """
    Fetch multiple IDs from the target_table based on matching_fields in the DataFrame.

    Parameters:
    - conn: Database connection object
    - df: DataFrame containing the fields to match
    - target_table: The name of the table to query (e.g., 'booking_places')
    - target_fields: A list of fields to retrieve from the target_table (e.g., ['place_id', 'city_entityid'])
    - matching_fields: A dictionary where keys are field names in the target_table and values are column names in the df

    Returns:
    - A list of tuples with IDs retrieved from the database
    """
    cursor = conn.cursor()
    results = []
    
    # Construct SELECT clause and WHERE clause
    select_clause = ", ".join(target_fields)
    where_clause = " AND ".join([f"{table_field} = %s" for table_field in matching_fields.keys()])
    query = f"SELECT {select_clause} FROM {target_table} WHERE {where_clause};"
    
    for _, row in df.iterrows():
        values = tuple(row[df_field] for df_field in matching_fields.values())
        
        cursor.execute(query, values)
        result = cursor.fetchone()
        results.append(result if result else (None,) * len(target_fields))

    cursor.close()
    return results


In [2911]:
accommodation_prices["city"] = accommodation_prices["city"].map(city_entityid_map)

In [2912]:
import warnings
warnings.filterwarnings("ignore")

In [2913]:
# Define the fields you want to retrieve and the matching criteria
target_fields = ["place_id"]
matching_fields = {
    "city_entityid": "city",
    "name": "name"  
}

# Call the function and add the result to your DataFrame
results = fetch_multiple_ids_from_db(conn, accommodation_prices, "booking_places", target_fields, matching_fields)
accommodation_prices[['place_id']] = pd.DataFrame(results, index=accommodations.index)

In [2914]:
# Define the fields you want to retrieve and the matching criteria
target_fields = ["accommodation_id"]
matching_fields = {
    "room_type": "room_type",
    "place_id": "place_id"  
}

# Call the function and add the result to your DataFrame
results = fetch_multiple_ids_from_db(conn, accommodation_prices, "accommodations", target_fields, matching_fields)
accommodation_prices[['accommodation_id']] = pd.DataFrame(results, index=accommodations.index)
accommodation_prices.drop(columns="place_id",inplace=True)

In [2915]:
accommodation_prices.head(2)

,city,name,room_type,query_date,checkin,checkout,n_adults_search,n_children_search,n_rooms_search,price_night,price_currency,free_cancellation,pay_at_hotel,free_taxi,accommodation_id
0,27548283,Nice and comfortable room for your stay in BCN,Habitación Doble Económica,2024-11-04 00:43:02.045730,2024-11-08,2024-11-10,2,0,1,74.0,€,False,False,False,1.0
1,27548283,Hotel Derby,Habitación (1 o 2 adultos) - 1 o 2 camas,2024-11-04 00:43:02.239603,2024-11-08,2024-11-10,2,0,1,136.0,€,False,False,False,2.0


In [2916]:
create_accommodation_prices = """
CREATE TABLE accommodation_prices (
    price_id SERIAL PRIMARY KEY,
    accommodation_id INT REFERENCES accommodations(accommodation_id),
    query_date TIMESTAMP NOT NULL,
    checkin DATE NOT NULL,
    checkout DATE NOT NULL,
    n_adults INT NOT NULL,
    n_children INT DEFAULT 0,
    n_rooms INT NOT NULL,
    price_night NUMERIC NOT NULL,
    price_currency VARCHAR(4) NOT NULL,
    free_cancellation BOOLEAN,
    pay_at_hotel BOOLEAN,
    free_taxi BOOLEAN
);
"""

In [2917]:
accommodation_prices = accommodation_prices.iloc[:,3:]
accommodation_prices.head(1)

,query_date,checkin,checkout,n_adults_search,n_children_search,n_rooms_search,price_night,price_currency,free_cancellation,pay_at_hotel,free_taxi,accommodation_id
0,2024-11-04 00:43:02.045730,2024-11-08,2024-11-10,2,0,1,74.0,€,False,False,False,1.0


In [2918]:
accommodation_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22980 entries, 0 to 22979
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   query_date         22980 non-null  datetime64[ns]
 1   checkin            22980 non-null  datetime64[ns]
 2   checkout           22980 non-null  datetime64[ns]
 3   n_adults_search    22980 non-null  object        
 4   n_children_search  22980 non-null  object        
 5   n_rooms_search     22980 non-null  object        
 6   price_night        22980 non-null  float64       
 7   price_currency     22980 non-null  object        
 8   free_cancellation  22980 non-null  bool          
 9   pay_at_hotel       22980 non-null  bool          
 10  free_taxi          22980 non-null  bool          
 11  accommodation_id   22976 non-null  float64       
dtypes: bool(3), datetime64[ns](3), float64(2), object(4)
memory usage: 1.6+ MB


In [2919]:
create_accommodation_prices = """
CREATE TABLE accommodation_prices (
    price_id SERIAL PRIMARY KEY,
    accommodation_id INT REFERENCES accommodations(accommodation_id),
    query_date TIMESTAMP NOT NULL,
    checkin DATE NOT NULL,
    checkout DATE NOT NULL,
    n_adults INT NOT NULL,
    n_children INT DEFAULT 0,
    n_rooms INT NOT NULL,
    price_night NUMERIC NOT NULL,
    price_currency VARCHAR(4) NOT NULL,
    free_cancellation BOOLEAN,
    pay_at_hotel BOOLEAN,
    free_taxi BOOLEAN
);
"""

In [2925]:
accommodation_prices[["n_adults_search","n_children_search","n_rooms_search"]] = accommodation_prices[["n_adults_search","n_children_search","n_rooms_search"]].astype(int)

In [2926]:
accommodation_prices.head(1)

,query_date,checkin,checkout,n_adults_search,n_children_search,n_rooms_search,price_night,price_currency,free_cancellation,pay_at_hotel,free_taxi,accommodation_id
0,2024-11-04 00:43:02.045730,2024-11-08,2024-11-10,2,0,1,74.0,€,False,False,False,1.0


In [2932]:
accommodation_prices = accommodation_prices[~accommodation_prices["accommodation_id"].isna()]

In [2933]:
conn.cursor().executemany(
    """
    INSERT INTO accommodation_prices ( 
        query_date, 
        checkin, 
        checkout,
        n_adults,
        n_children,
        n_rooms,
        price_night,
        price_currency,
        free_cancellation,
        pay_at_hotel,
        free_taxi,
        accommodation_id
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """,
    list(accommodation_prices.itertuples(index=False, name=None))
)


### 3.3.8 Activities

In [3034]:

create_activities = """
CREATE TABLE activities (
    activity_id SERIAL PRIMARY KEY,
    activity_name VARCHAR(500) NOT NULL,
    city_entityid INT REFERENCES cities(city_entityid) ON DELETE SET NULL,
    description TEXT,
    url VARCHAR(500),
    image VARCHAR(500),
    duration VARCHAR(30),
    latitude NUMERIC,
    longitude NUMERIC,
    category VARCHAR(100),
    spanish VARCHAR(30),
    address VARCHAR(500)
);
"""

In [3035]:
activities_table.head(1)

,activity_name,city,description,url,image,duration,latitude,longitude,category,spanish,address
0,Excursión a Montserrat + Visita a una bodega,barcelona,En esta excursión a Montserrat no solo disfrut...,www.civitatis.com/es/barcelona/tour-tapas-vino...,www.civitatis.com/f/espana/barcelona/tour-tapa...,7h 30m,41.3940236912484,2.181866082214644,Gastronomía y enoturismo,Español,"Barcelona (Bus Terminal Nord), Eixample, 08013..."


In [ ]:
activities_table["city"].replace("sevilla","seville",inplace=True)
activities_table["city"] = activities_table["city"].map(city_entityid_map)

In [3037]:
conn.cursor().executemany(
    """
    INSERT INTO activities ( 
        activity_name, 
        city_entityid, 
        description,
        url,
        image,
        duration,
        latitude,
        longitude,
        category,
        spanish,
        address
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """,
    list(activities_table.itertuples(index=False, name=None))
)


### 3.3.9 Activities prices

In [3038]:
create_activity_prices = """
CREATE TABLE activity_prices (
    price_id SERIAL PRIMARY KEY,
    activity_id INT REFERENCES activities(activity_id) ON DELETE CASCADE,
    query_date DATE NOT NULL,
    price NUMERIC NOT NULL,
    currency VARCHAR(4) NOT NULL
);
"""


In [3047]:
activities_prices["city"].replace("sevilla","seville",inplace=True)
activities_prices["city"] = activities_prices["city"].map(city_entityid_map)

In [3048]:
activities_prices.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15420 entries, 0 to 18450
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   activity_name  15420 non-null  object        
 1   city           15420 non-null  int64         
 2   query_date     15420 non-null  datetime64[ns]
 3   price          15361 non-null  float64       
 4   currency       15361 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 722.8+ KB


In [3049]:
# Define the fields you want to retrieve and the matching criteria
target_fields = ["activity_id"]
matching_fields = {
    "activity_name": "activity_name",
    "city_entityid": "city"  
}

# Call the function and add the result to your DataFrame
results = fetch_multiple_ids_from_db(conn, activities_prices, "activities", target_fields, matching_fields)
activities_prices[['activity_id']] = pd.DataFrame(results, index=activities_prices.index)
activities_prices = activities_prices[["query_date","price","activity_id"]]

In [3050]:
conn.cursor().executemany(
    """
    INSERT INTO activity_prices ( 
        query_date, 
        price,
        activity_id
    ) VALUES (%s, %s, %s)
    """,
    list(activities_prices.itertuples(index=False, name=None))
)

### 3.3.10 Activities availabilities

In [3051]:
create_activity_availabilities = """
CREATE TABLE activity_availabilities (
    schedule_id SERIAL PRIMARY KEY,
    activity_id INT REFERENCES activities(activity_id) ON DELETE CASCADE,
    query_date TIMESTAMP NOT NULL,
    available_date DATE NOT NULL,
    available_time TIME NOT NULL
);
"""

In [3052]:
activities_schedules["city"].replace("sevilla","seville",inplace=True)
activities_schedules["city"] = activities_schedules["city"].map(city_entityid_map)

In [3053]:
# Define the fields you want to retrieve and the matching criteria
target_fields = ["activity_id"]
matching_fields = {
    "activity_name": "activity_name",
    "city_entityid": "city"  
}

# Call the function and add the result to your DataFrame
results = fetch_multiple_ids_from_db(conn, activities_schedules, "activities", target_fields, matching_fields)
activities_schedules[['activity_id']] = pd.DataFrame(results, index=activities_schedules.index)
activities_schedules = activities_schedules[["query_date","available_date","available_times","activity_id"]]

In [3054]:
activities_schedules.head(1)

,query_date,available_date,available_times,activity_id
0,2024-11-04 01:23:44.031007,2024-11-06,08:45:00,256


In [3055]:
conn.cursor().executemany(
    """
    INSERT INTO activity_availabilities ( 
        query_date, 
        available_date,
        available_time,
        activity_id
    ) VALUES (%s, %s, %s, %s)
    """,
    list(activities_schedules.itertuples(index=False, name=None))
)

### 3.3.11 Weather

In [3068]:
destinations = ['barcelona', 'malaga', 'seville', 'valencia']
weather = weather[weather["city"].isin(destinations)]

In [3069]:
weather["city"] = weather["city"].map(city_entityid_map)

In [3071]:
conn.cursor().executemany(
    """
    INSERT INTO weather_data (
        date, 
        apparent_temperature_mean, 
        apparent_temperature_min, 
        apparent_temperature_max, 
        precipitation_sum, 
        precipitation_hours, 
        wind_speed_10m_max, 
        wind_gusts_10m_max, 
        sunshine_duration, 
        daylight_duration, 
        city_entityid, 
        forecast_history
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """,
    list(weather.itertuples(index=False, name=None))
)
